In [1]:
from collections import defaultdict

# Is a triangulated surface orientable?

Each triangle is stored as a tuple of its vertices. Each vertex is labeled with a non negative integer.

A triangulation of a surface is stored as a simple graph. Each triangle in the triangulation is stored in the corresponding node in the graph. Two nodes in the graph are connected when the triangles stored inside them share a common edge.

Remark: a graph is cubic if a triangulated surface is 2D manifold without a boundary.

In [2]:
class Triangle:
    """
    A triangle is represented as a list of its
    vertices (labeled with natural numbers).
    """
    def __init__(self, vertices):
        assert len(vertices) == 3, 'A triangle should have 3 vertices'
        self.vertices = sorted(vertices)
        
    @property
    def edges(self):
        """
        Return the list of edges 'in the positive direction'.
        """
        return (
                (self.vertices[0], self.vertices[1]),
                (self.vertices[1], self.vertices[2]),
                (self.vertices[2], self.vertices[0]),
               )

    def contains_edge(self, edge):
        '''
        Return True, if the triangle contains a given edge. 
        The edge is given as a tuple.
        '''
        return edge[0] in self.vertices and edge[1] in self.vertices
    
    def __repr__(self):
        return str(self.vertices)

In [11]:
class Node:
    """
    A node in a graph.
    Each node stores a set of its neighbours and a pointer to the data stored in the node.
    """
    def __init__(self, data=None):
        self.neighbours = set()
        self.data = data

Example triangulations: a set of triangles that represents a torus and a projective plane.

In [4]:
torus_triangles = [
    (0, 1, 6), (0, 4, 6), (1, 6, 7), (1, 2, 7), (2, 7, 4), (2, 0, 4), (4, 5, 8), (4, 6, 8), (6, 3, 8),
    (6, 7, 3), (3, 7, 5), (7, 4, 5), (0, 1, 5), (1, 5, 8), (1, 2, 8), (2, 8, 3), (2, 0, 3), (0, 5, 3),
]

projective_plane_triangles = [
    (1, 2, 7), (1, 7, 3), (2, 7, 5), (7, 3, 6), (2, 3, 5), (4, 5, 7),
    (7, 4, 6), (2, 6, 3), (4, 5, 3), (4, 6, 2), (1, 3, 4), (4, 1, 2),
]

First we create a graph from the set of triangles. No ordered structure is computed here.

In [23]:
def create_triangulation(triangles):
    n = max([max(triangle) for triangle in triangles]) + 1
    nodes = [Node(Triangle(triangle)) for triangle in triangles]
    # For each vertex compute the set of nodes containing triangles containing the vertex.    
    vertex_nodes = defaultdict(set)
    for node in nodes:
        for vertex in node.data.vertices:
            vertex_nodes[vertex].add(node)
    # Make connection between neighbourhood nodes.
    for node in nodes:
        for (v1, v2) in node.data.edges:
            neighbour = (vertex_nodes[v1] & vertex_nodes[v2])
            neighbour.remove(node)
            assert len(neighbour) == 1
            node.neighbours.add(neighbour.pop())
        # Each node should have degree 3 (2D manifold without a boundary)
        assert len(node.neighbours) == 3
    return nodes    

An oriented version of a triangle is represented as a tuple (triangle, orientation). First we define helper functions enext and sym that move from one oriented triangle to another.

In [41]:
def sym(oriented_triangle):
    """
    Get the orientation with the same leading edge in the oposite direction.
    """
    triangle, orientation  = oriented_triangle
    return triangle, (orientation + 4) % 8

def enext(oriented_triangle):
    """
    Get the next orientation of the same type.
    """
    triangle, orientation = oriented_triangle
    if orientation <= 2:
        orientation = (orientation + 1) % 3
    else:
        orientation = ((orientation + 1) % 3) + 4
    return triangle, orientation

def fnext(oriented_triangle):
    edge = get_leading_edge(oriented_triangle)
    triangle, orientation = oriented_triangle
    # Get the neighbour that shares the common edge with our triangle
    neighbour = [n for n in neighbours if n.data.contains_edge(edge)][0]
    orientation = get_orientation(edge, neighbour.data.edges)
    ordered[i] = (neighbour.data, orientation)
    ordered[sym(i)] = (neighbour.data, sym(orientation))


def is_orientation_positive(orientation):
    if orientation in [0, 1, 2]: 
        return True
    return False

def get_leading_edge(oriented_triangle):
    triangle, orientation = oriented_triangle
    edge = triangle.edges[orientation & 3]
    if not is_orientation_positive(orientation):
        edge = tuple(reversed(edge))
    return edge
    

def get_orientation(edge, edges):
    """
    Returt the orientation of edge in the given set of edges.
    It can be 0, 1 or 2 (edge is one of the edges)
    or 4, 5 or 6 (edge with a positeva orientation is one of the edges).
    """
    try:
        index =  edges.index(edge)
    except Exception:
        index = edges.index(tuple(reversed(edge))) + 4
    return index
    
def order_triangulation(triangulation):
    """
    Construct ordered triangles for a given triangulation.
    """
    for node in triangulation:
        triangle = node.data
        neighbours = node.neighbours
        # Two assumptions made here:
        # - triangle vertices are ordered
        # - function edges of triangle returns edges  
        #   in the positive orientation
        ordered = dict()
        for i, edge in enumerate(triangle.edges):
            # Get the neighbour that shares the common edge with our triangle
            neighbour = [n for n in neighbours if n.data.contains_edge(edge)][0]
            orientation = get_orientation(edge, neighbour.data.edges)
            ordered[i] = (neighbour.data, orientation)
            ordered[sym(i)] = (neighbour.data, sym(orientation))
        triangle.ordered = ordered

In [42]:
tuple(reversed((1, 2)))


(2, 1)

Finally: a function which checks whether a given triangulation is orientable.

In [43]:
def orientable(triangles):
    '''
    Is a triangulation orientable?
    '''
    triangulation = create_triangulation(triangles)
    order_triangulation(triangulation)
    return is_orientable(triangulation[0].data.ordered[0])

def same_orientation_type(o1, o2):
    '''
    Are two orientations of the same type.
    '''
    assert o1 in [0, 1, 2, 4, 5, 6]
    assert o2 in [0, 1, 2, 4, 5, 6]
    return o1 >> 2 == o2 >> 2

def is_orientable(oriented_triangle):
    (triangle, orientation) = oriented_triangle
    chosen_orientation = getattr(triangle, 'orientation', None)
    if chosen_orientation is None:
        triangle.orientation = orientation
        (t1, o1) = triangle.ordered[sym(orientation)]
        (t2, o2) = triangle.ordered[enext(sym(orientation))]
        (t3, o3) = triangle.ordered[enext(enext(sym(orientation)))]
        return is_orientable((t1, o1)) and is_orientable((t2, o2)) and is_orientable((t3, o3))
    else:
        return same_orientation_type(orientation, chosen_orientation)

In [44]:
orientable(torus_triangles)

TypeError: 'int' object is not iterable

In [45]:
orientable(projective_plane_triangles)

TypeError: 'int' object is not iterable